In [1]:

import pandas as pd
import numpy as np
import nltk
from bs4 import BeautifulSoup as bs
#import scrapy
#from scrapy.crawler import CrawlerProcess
import requests
import re
import urllib.request
import os
import glob
import tensorflow as tf
import keras

Using TensorFlow backend.


In [2]:
trainpath = "dataset/train/processed_train.csv"
valpath = "dataset/train/processed_val.csv"


In [3]:
df_val = pd.read_csv(valpath)
df_val.head(1)

,Unnamed: 0,label,text
0,0,12,How do I stop getting offended.reactive when p...


In [4]:
df_train = pd.read_csv(trainpath)
df_train.head(1)



,Unnamed: 0,label,text
0,0,0,"[oops ignore, delete]...Noordenwind Yes, I am ..."


In [5]:
df_train = df_train[["label","text"]]
df_train.head()

,label,text
0,0,"[oops ignore, delete]...Noordenwind Yes, I am ..."
1,1,ISTP Women Straight hotties. Im an ESTP male a...
2,2,"Yup, thats small talk in a nutshell....https....."
3,3,Perhaps you are depressed. Also I would defini...
4,4,Anyone working on long-term projects Just curi...


In [6]:
df_val = df_val[["label","text"]]
df_val.head(1)


,label,text
0,12,How do I stop getting offended.reactive when p...


In [7]:
print(df_train.shape)
print(df_val.shape)


(16140, 2)
(1798, 2)


In [29]:
labels = list(df_train['label'].unique())
print(labels)

['infp', 'istp', 'intj', 'intp', 'estj', 'entp', 'isfj', 'istj', 'isfp', 'infj', 'enfp', 'entj', 'enfj', 'esfj', 'esfp', 'estp']


In [31]:
for i in range(df_train.shape[0]):
    df_train["label"][i] = labels.index(str(df_train["label"][i]))
df_train.head()

,label,text
0,0,"[oops ignore, delete]...Noordenwind Yes, I am ..."
1,1,ISTP Women Straight hotties. Im an ESTP male a...
2,2,"Yup, thats small talk in a nutshell....https....."
3,3,Perhaps you are depressed. Also I would defini...
4,4,Anyone working on long-term projects Just curi...


In [33]:
df_train["label"].value_counts()

9     1820
0     1760
2     1562
3     1265
5     1073
10    1023
1     1005
11    1000
8      996
7      962
12     945
6      915
15     590
14     495
4      399
13     330
Name: label, dtype: int64

In [34]:
df_val["label"].value_counts()



infj    217
intj    192
intp    165
infp    162
entj    128
entp    119
istp    112
isfp    110
enfp    109
isfj    102
istj     97
enfj     87
estp     64
estj     50
esfj     49
esfp     35
Name: label, dtype: int64

In [35]:
for i in range(df_val.shape[0]):
    df_val["label"][i] = labels.index(str(df_val["label"][i]))
df_val["label"].value_counts()


9     217
2     192
3     165
0     162
11    128
5     119
1     112
8     110
10    109
6     102
7      97
12     87
15     64
4      50
13     49
14     35
Name: label, dtype: int64

In [39]:
df_train.head(1)

,label,text
0,0,"[oops ignore, delete]...Noordenwind Yes, I am ..."


In [41]:
df_train.to_csv(trainpath)
df_val.to_csv(valpath)


In [45]:
df = pd.read_csv(trainpath)
df.head()

,Unnamed: 0,label,text
0,0,0,"[oops ignore, delete]...Noordenwind Yes, I am ..."
1,1,1,ISTP Women Straight hotties. Im an ESTP male a...
2,2,2,"Yup, thats small talk in a nutshell....https....."
3,3,3,Perhaps you are depressed. Also I would defini...
4,4,4,Anyone working on long-term projects Just curi...


In [48]:
df["label"].value_counts()

9     1820
0     1760
2     1562
3     1265
5     1073
10    1023
1     1005
11    1000
8      996
7      962
12     945
6      915
15     590
14     495
4      399
13     330
Name: label, dtype: int64

In [7]:
df_train.head(1)


,label,text
0,0,"[oops ignore, delete]...Noordenwind Yes, I am ..."


In [8]:
x_train = []
max_len = 0
for i in range(df_train.shape[0]):
    x_train.append(str(df_train["text"][i]))
    if len(df_train["text"][i]) > max_len:
        max_len = len(df_train["text"][i])
print(len(x_train))
print(max_len)

16140
24412


In [12]:
x_val = []
max_len_val = 0
for i in range(df_val.shape[0]):
    x_val.append(str(df_val["text"][i]))
    if len(df_val["text"][i]) > max_len_val:
        max_len_val = len(df_val["text"][i])
print(len(x_val))
print(max_len_val)



1798
15127


In [15]:
print(len(x_train))
print(len(x_val))

16140
1798


In [16]:
type(df_train["label"][0])

numpy.int64

In [17]:
y_train = np.array(df_train["label"])
y_train



array([ 0,  1,  2, ..., 10,  6, 15])

In [18]:
y_val = np.array(df_val["label"])
y_val


array([12, 10,  9, ...,  3,  9,  7])

In [19]:
vocab_size = 10000
embedding_dim = 16
max_length = 20000
trunc_type='post'
oov_tok = "<OOV>"

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(x_train)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(x_train)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(x_val)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)


In [20]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[1]))
print(x_train[1])



? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? 

In [21]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32)),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20000, 16)         160000    
_________________________________________________________________
bidirectional (Bidirectional (None, 64)                9600      
_________________________________________________________________
dense (Dense)                (None, 6)                 390       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 169,997
Trainable params: 169,997
Non-trainable params: 0
_________________________________________________________________


In [ ]:
num_epochs = 50
history = model.fit(padded, y_train, epochs=num_epochs, validation_data=(testing_padded, y_val))

